# Tensorflow 实现 Bidirectional LSTM Classifier

此处使用tensorflow实现一个Bidirectional LSTM Classifier， 并在MNIST数据集上进行测试

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot = True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.01
max_samples = 400000
batch_size = 128
display_step = 10

n_input = 28
n_steps = 28
n_hidden = 256
n_classes = 10

In [3]:
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

weighs = tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
biases = tf.Variable(tf.random_normal([n_classes]))

In [4]:
def BiRNN(x, weighs, biases):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_steps)
    
    lstm_fw_cell =  tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weighs) + biases

In [5]:
pred = BiRNN(x, weighs, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [6]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    step =1
    while step*batch_size <max_samples:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape(batch_size, n_steps, n_input)
        sess.run(optimizer,feed_dict={x: batch_x, y:batch_y})
        if step%display_step == 0:
            acc = sess.run(accuracy, feed_dict={x:batch_x, y:batch_y})
            loss = sess.run(cost, feed_dict={x:batch_x, y:batch_y})
            print("Iter"+str(step*batch_size) + ", Minibatch Loss = "+ "{:.6f}".format(loss) + ", Training Accuracy=" + "{:.5f}".format(acc))
        step += 1
        
    print("Optimizer Finished!")
    
    test_len = 10000
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Test Accuracy: " , sess.run(accuracy, feed_dict={x:test_data, y:test_label}))

Iter1280, Minibatch Loss = 1.784165, Training Accuracy=0.40625
Iter2560, Minibatch Loss = 1.234419, Training Accuracy=0.60938
Iter3840, Minibatch Loss = 0.989624, Training Accuracy=0.68750
Iter5120, Minibatch Loss = 0.606591, Training Accuracy=0.77344
Iter6400, Minibatch Loss = 0.494237, Training Accuracy=0.77344
Iter7680, Minibatch Loss = 0.878752, Training Accuracy=0.71875
Iter8960, Minibatch Loss = 0.446996, Training Accuracy=0.88281
Iter10240, Minibatch Loss = 0.287107, Training Accuracy=0.90625
Iter11520, Minibatch Loss = 0.308100, Training Accuracy=0.89844
Iter12800, Minibatch Loss = 0.367769, Training Accuracy=0.89844
Iter14080, Minibatch Loss = 0.193730, Training Accuracy=0.93750
Iter15360, Minibatch Loss = 0.209150, Training Accuracy=0.92969
Iter16640, Minibatch Loss = 0.428895, Training Accuracy=0.87500
Iter17920, Minibatch Loss = 0.277923, Training Accuracy=0.90625
Iter19200, Minibatch Loss = 0.152714, Training Accuracy=0.96094
Iter20480, Minibatch Loss = 0.076126, Training 